In [31]:
import os

import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization
from tensorflow.keras.models import load_model, Model

from attention import Attention
from load import load_data
from aggregate import Aggregate

tra_date = '2007-02-14'
val_date = '2015-01-02'
tes_date = '2016-01-04'
model_selection = 'dtml'

index_data_path = './data/kdd17/index/'
index_fname = 'SPY_processed.csv'
data_path = './data/kdd17/processed/'
num_tickers = 50
params = {
    'input_dim': 11,
    'time_steps': 15,
    'epochs': 5,
}

fnames = [fname for fname in os.listdir(data_path) if
            os.path.isfile(os.path.join(data_path, fname))]
print(len(fnames), ' tickers')

stock_layers = [x[:-4] for x in fnames]


stock_context_vector = np.zeros([50, 1970, 11], dtype=float)
index_context_vector = np.zeros([1, 1970, 11], dtype=float)

stock_model = []
aggreated_model = []

def time_axis_attention(data_path, fname):
    print(fname)
    tra_pv, tra_wd, tra_gt, val_pv, val_wd, val_gt, tes_pv, tes_wd, tes_gt = load_data(data_path, fname,
            tra_date, val_date, tes_date, seq=params['time_steps']
        ) 
    model_input = Input(shape=(params['time_steps'], params['input_dim']))
    x = Dense(params['input_dim'])(model_input)
    x = LSTM(64, return_sequences=True)(x)
    x = Attention(params['input_dim'])(x)
    x = BatchNormalization()(x)
    model = Model(model_input, x)
    model.compile(loss='mae', optimizer='adam')
    # context vector
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    intermediate_output = intermediate_layer_model(tra_pv).numpy()
    return intermediate_output

    # x = Dense(1, activation = 'sigmoid')(x)
    # x = Dense(1)(x)
    # model = Model(model_input, x)
    # model.compile(loss='mae', optimizer='adam')
    # model.summary()

    # save context vector
    # intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    # intermediate_output = intermediate_layer_model(tra_pv)
    # stock_context_vector[idx] = intermediate_output.numpy()

    # model.fit(tra_pv, tra_gt, epochs=params['epochs'])

49  tickers


In [32]:
index_context_vector = time_axis_attention(index_data_path, index_fname)
for idx, fname in enumerate(fnames):
    stock_context_vector[idx] = time_axis_attention(data_path, fname)    

SPY_processed.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
252  testing instances
BA.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
TM.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
MO.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
NVS.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
T.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation instances
data_EOD 2518
251  testing instances
DCM.csv
1  tickers selected
2518 trading dates:
29 2014 2266
1970  training instances
252  validation

In [ ]:
def get_attention_model():
    model_input = Input(shape=(params['time_steps'], params['input_dim']))
    x = Dense(params['input_dim'])(model_input)
    x = LSTM(64, return_sequences=True)(x)
    x = Attention(params['input_dim'])(x)
    x = BatchNormalization()(x)
    return model


In [26]:
index_model = get_attention_model()

model_input = Input(shape=(params['time_steps'], params['input_dim']))
for idx, fname in fnames:
    stock_model[idx] = get_attention_model()
    aggreated_model[idx] = Model(model_input, x)
    aggreated_model[idx] = Aggregate(params['input_dim'])([index_model, stock_model])
    stock_model[idx].compile(loss='mae', optimizer='adam')

array([[-0.01392356, -0.00018619, -0.00656118, ...,  0.00435714,
        -0.01140799,  0.00162378],
       [-0.01207639, -0.00022943, -0.00734343, ...,  0.0032198 ,
        -0.0103576 ,  0.00117521],
       [-0.01013519, -0.00050935, -0.00874232, ...,  0.00221726,
        -0.00900753,  0.00061732],
       ...,
       [ 0.00362593,  0.00111953, -0.00141175, ..., -0.00125042,
         0.00311573, -0.00240044],
       [ 0.00325921,  0.00122984, -0.00060503, ..., -0.00086837,
         0.00203469, -0.00156328],
       [ 0.00216491,  0.00141626,  0.00057647, ..., -0.00015327,
         0.000624  , -0.00071362]])

In [ ]:
# context vector
for idx, fname in fnames:
    model = stock_model[idx]
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    intermediate_output = intermediate_layer_model(tra_pv).numpy()

In [14]:
# def context_aggregation():
#     for stock in stock_context_vector:
#         merged = Aggregate(index.shape[1])([index, stock])
Aggregate(11)([index, apple])

<tf.Tensor: shape=(1970, 11), dtype=float32, numpy=
array([[ 5.4233335e-03,  2.4390398e-03,  2.3545958e-03, ...,
        -7.7093550e-04,  5.6178048e-03,  2.4059550e-03],
       [ 4.6143364e-03,  2.5975327e-03,  2.0954572e-03, ...,
        -4.4679292e-04,  5.4750405e-03,  2.8891736e-03],
       [ 4.0299036e-03,  2.5541140e-03,  1.9977712e-03, ...,
        -3.4593028e-04,  5.4613161e-03,  3.3186213e-03],
       ...,
       [ 3.2325275e-04, -4.8872544e-03, -1.0408924e-02, ...,
         2.0146335e-03, -9.7539620e-03, -1.1712414e-03],
       [ 1.5361602e-03, -3.7946980e-03, -1.1651803e-02, ...,
         2.6936331e-03, -1.0067202e-02,  9.5471914e-06],
       [ 2.4166857e-03, -2.8484033e-03, -1.0264205e-02, ...,
         2.5905017e-03, -8.8460771e-03, -1.9430008e-06]], dtype=float32)>

In [ ]:
fnames = [fname for fname in os.listdir(data_path) if
            os.path.isfile(os.path.join(data_path, fname))]
print(len(fnames), ' tickers')


for idx, fname in enumerate(fnames):
    print(fname)
    tra_pv, tra_wd, tra_gt, val_pv, val_wd, val_gt, tes_pv, tes_wd, tes_gt = load_data(data_path, fname,
            tra_date, val_date, tes_date, seq=params['time_steps']
        ) 
    model_input = Input(shape=(params['time_steps'], params['input_dim']))
    x = Dense(params['input_dim'])(model_input)
    x = LSTM(64, return_sequences=True)(x)
    x = Attention(params['input_dim'])(x)
    # x = Dense(1, activation = 'sigmoid')(x)
    # x = Dense(1)(x)
    model = Model(model_input, x)
    model.compile(loss='mae', optimizer='adam')
    model.summary()

    # save context vector
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
    intermediate_output = intermediate_layer_model(tra_pv)
    stock_context_vector[idx] = intermediate_output.numpy()

    model.fit(tra_pv, tra_gt, epochs=params['epochs'])


In [19]:
multi_level_context_vector

array([[[-3.94268474e-03, -7.68327503e-04, -7.77846617e-06, ...,
          2.55864183e-03,  6.68808771e-03, -5.28394943e-03],
        [-3.91988689e-03, -1.85969948e-05,  2.78514519e-04, ...,
          1.62527629e-03,  5.63550927e-03, -4.83782729e-03],
        [-3.79534112e-03,  1.00350915e-03,  9.82391415e-04, ...,
          5.01917093e-04,  4.61288402e-03, -4.38891398e-03],
        ...,
        [-1.74346031e-03,  7.89974071e-03,  4.27981466e-03, ...,
          3.17770580e-04, -2.28511728e-03, -3.51359812e-03],
        [-2.32273655e-04,  7.64224492e-03,  4.28253319e-03, ...,
         -1.69997814e-03, -5.04839048e-03, -1.16537570e-03],
        [ 1.41247991e-03,  6.23681024e-03,  3.40810209e-03, ...,
         -2.51824432e-03, -7.17436662e-03,  1.18894898e-03]],

       [[-3.23717296e-03, -3.93352332e-03, -9.29951435e-04, ...,
          3.58164334e-03,  9.75792296e-03, -3.78302950e-03],
        [-3.66728404e-03, -3.00392136e-03,  1.05192745e-03, ...,
          1.20060355e-03,  7.98861310e

In [126]:
# get context vector for index
tra_pv, tra_wd, tra_gt, val_pv, val_wd, val_gt, tes_pv, tes_wd, tes_gt = load_data(data_path, fname,
        tra_date, val_date, tes_date, seq=params['time_steps']
    ) 
model_input = Input(shape=(params['time_steps'], params['input_dim']))
x = Dense(params['input_dim'])(model_input)
x = LSTM(64, return_sequences=True)(x)
x = Attention(params['input_dim'])(x)
# x = Dense(1, activation = 'sigmoid')(x)
# x = Dense(1)(x)
model = Model(model_input, x)
model.compile(loss='mae', optimizer='adam')
model.summary()

# save context vector
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-1].output)
intermediate_output = intermediate_layer_model(tra_pv)
multi_level_context_vector[idx] = intermediate_output.numpy()

model.fit(tra_pv, tra_gt, epochs=params['epochs'])

[]
(1, 3)
(2, 3)
[[1. 2. 3.]
 [4. 5. 0.]]


In [128]:
arr = np.empty((0,3))
print(arr)
arr = np.append(arr, np.array([[1, 2, 3]]), axis=0)
arr = np.append(arr, np.array([[4, 5, 0]]), axis=0)
print(arr)

[]
[[1. 2. 3.]
 [4. 5. 0.]]


In [53]:
np.zeros([2,3])

array([[0., 0., 0.],
       [0., 0., 0.]])